In [1]:
import time
import pandas as pd
import numpy as np
import math
from math import cos, asin, sqrt
import gc
pd.set_option('display.max_columns', None)


In [2]:
#ejecutar esto solo la primera vez#####################################################
start = time.time()
vuelos = pd.read_csv('vuelosDeparted.csv', sep=',',low_memory=False, )
vuelos["TMIN_o"]= ''
vuelos["TMAX_o"]= ''
vuelos["TAVG_o"]= ''
vuelos["SNOW_o"]= ''
vuelos["PRCP_o"]= ''
vuelos["TMIN_d"]= ''
vuelos["TMAX_d"]= ''
vuelos["TAVG_d"]= ''
vuelos["SNOW_d"]= ''
vuelos["PRCP_d"]= ''
vuelos.to_csv('vuelos.csv', sep=',', index=False)
vuelos = pd.read_csv('vuelos.csv', sep=',',low_memory=False, )
vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
end = time.time()
print(end - start)

10.7059998512


In [3]:
# Load of geobase
start = time.time()
from GeoBases import GeoBase
geo_c = GeoBase(data='ori_por', verbose=False)
stationstxt = ""
with open("stations.txt") as input:
    stationstxt = input.read()
    
#Extract the data from file
stations2 = stationstxt.split("\n")
#Remove last line
stations2 = stations2[:-1]
stations2 = map(lambda line: [line[0:11],float(line[13:20]),float(line[22:30]),line[41:71]], stations2)

#weather2012 = pd.read_csv("2012.csv",header=None)
#weather2012.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]

#weather2011 = pd.read_csv("2011.csv",header=None)
#weather2011.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
#weather2014 = pd.read_csv("2014.csv",header=None)
#weather2014.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
weather2015 = pd.read_csv("2015.csv",header=None)
weather2015.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
#weather2016 = pd.read_csv("2016.csv",header=None)
#weather2016.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
#weather2017 = pd.read_csv("2017.csv",header=None)
#weather2017.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
end = time.time()
print(end - start)

40.1979999542


In [4]:
#del weather2011
del weather2014
#del weather2012
gc.collect()

NameError: name 'weather2014' is not defined

In [5]:
stationsdf = pd.DataFrame(stations2)
stationsdf.columns = ["id","lat","lng","name"]
lats = stationsdf[['lat','lng']]

In [6]:
#funciones para calcular la distancia entre los aeropuertos y estacion metereologica mas cercana
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v[0],v[1],p[0],p[1]))

#para que la fecha no de problemas al comparar (quizas no este bien del todo)
def tratarFecha(fecha):
    fecha = str(fecha).split(' ')[0]
    fecha = fecha.replace('-','')
    fecha = int(fecha)
    return fecha

In [7]:
#para seleccionar con que año trabajar
weatherdf = weather2015

In [8]:
del weatherdf['Value3']
del weatherdf['Value4']
del weatherdf['Value5']
del weatherdf['Value2']

In [9]:
def tratar2(row):
    lat = row['board_lat']
    lon = row['board_lon']
    fecha = row['actual_time_of_departure']
    lat_d = row['off_lat']
    lon_d = row['off_lon']
    fecha_d = row['actual_time_of_arrival'] 
    ##################################
    #primero tratamos las fechas
    fecha = tratarFecha(fecha)
    fecha_d = tratarFecha(fecha_d)
    ##################################
    coor = [lat,lon]
    coor_off = [lat_d,lon_d]
    a= closest(np.asarray(lats), coor)
    b= closest(np.asarray(lats), coor_off)
    ##################################
    aux = stationsdf[stationsdf['lat']==a[0]]
    aux = stationsdf[stationsdf['lng']==a[1]]
    stationid = aux['id']
    s =stationid.to_string()
    stationid = str(s)
    stationid = stationid.split()[1]
    ##################################
    aux1 = stationsdf[stationsdf['lat']==b[0]]
    aux1 = stationsdf[stationsdf['lng']==b[1]]
    stationidoff = aux1['id']
    s =stationidoff.to_string()
    stationidoff = str(s)
    stationidoff = stationidoff.split()[1]
    #####################################
    data_board = weatherdf[(weatherdf["id"]==stationid)&(weatherdf['date']==fecha)].sort_values("date")
    data_off   = weatherdf[(weatherdf["id"]==stationidoff)&(weatherdf['date']==fecha_d)].sort_values("date")
    #######################################
    data_board["Value1"]=data_board["Value1"]/10
    data_board = data_board.reset_index()
    T1 = data_board[data_board['type']=='TAVG']['Value1']
    T2 = data_board[data_board['type']=='PRCP']['Value1']
    T3 = data_board[data_board['type']=='SNOW']['Value1']
    T4 = data_board[data_board['type']=='TMIN']['Value1']
    T5 = data_board[data_board['type']=='TMAX']['Value1']
    ##############################
    data_off["Value1"]=data_off["Value1"]/10
    data_off = data_off.reset_index()
    T6 = data_off[data_off['type']=='TAVG']['Value1']
    T7 = data_off[data_off['type']=='PRCP']['Value1']
    T8 = data_off[data_off['type']=='SNOW']['Value1']
    T9 = data_off[data_off['type']=='TMIN']['Value1']
    T10 = data_off[data_off['type']=='TMAX']['Value1']
    ##tratamos los valores nulos aplicando un valor nulo
    if T1.empty:
        T1 = 9999
    if T2.empty:
        T2 = 9999        
    if T3.empty:
        T3 = 9999        
    if T4.empty:
        T4 = 9999       
    if T5.empty:
        T5 = 9999
    if T6.empty:
        T6 = 9999      
    if T7.empty:
        T7 = 9999         
    if T8.empty:
        T8 = 9999        
    if T9.empty:
        T9 = 9999        
    if T10.empty:
        T10 = 9999        
        
    #actualizamos la fila
    row['TAVG_o']= float(T1)
    row['PRCP_o']= float(T2)
    row['SNOW_o']= float(T3)
    row['TMIN_o']= float(T4)
    row['TMAX_o']= float(T5)
    row['TAVG_d']= float(T6)
    row['PRCP_d']= float(T7)
    row['SNOW_d']= float(T8)
    row['TMIN_d']= float(T9)
    row['TMAX_d']= float(T10)
    return row

In [10]:
#para saber que vuelos tenemos de que años
vuelos2017 = vuelos[vuelos['anyoSalida']==2017]
vuelos2016 = vuelos[vuelos['anyoSalida']==2016]
vuelos2015 = vuelos[vuelos['anyoSalida']==2015]
vuelos2014 = vuelos[vuelos['anyoSalida']==2014]
vuelos2013 = vuelos[vuelos['anyoSalida']==2013]
vuelos2012 = vuelos[vuelos['anyoSalida']==2012]
vuelos2011 = vuelos[vuelos['anyoSalida']==2011]
vuelos2010 = vuelos[vuelos['anyoSalida']==2010]
vuelos2009 = vuelos[vuelos['anyoSalida']==2009]

#ver las dimensiones de cada año
print vuelos2017.shape
print vuelos2016.shape
print vuelos2015.shape
print vuelos2014.shape
print vuelos2013.shape
print vuelos2012.shape
print vuelos2011.shape
print vuelos2010.shape
print vuelos2009.shape

(24256, 66)
(146306, 66)
(41576, 66)
(556, 66)
(0, 66)
(3, 66)
(1, 66)
(0, 66)
(0, 66)


In [30]:
#vuelos = pd.read_csv('vuelos.csv', sep=',',low_memory=False, )
#vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
vuelosSinTratar = vuelos[vuelos['TAVG_o'].isnull()]
vuelosTratados = vuelos[vuelos['TAVG_o'].notnull()]
print vuelosSinTratar.shape
print vuelosTratados.shape
x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==2015]
print x.shape
x = x[x['mesSalida']==1]
print x.shape
x = x[x['diaSalida']==7]
print x.shape

(212693, 66)
(5, 66)
(41571, 66)
(3199, 66)
(98, 66)


In [22]:
def functionWeatherDay(month, day):
    u = int(month+day)
    weatherdf = weather2015
    weatherdf = weatherdf[weatherdf['date']-20150000==u]
    return weatherdf

In [37]:
def functionPlus(year, month, day):    
    start = time.time()
    #vuelos = pd.read_csv('vuelos.csv', sep=',',low_memory=False, )
    #vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    vuelosSinTratar = vuelos[vuelos['TAVG_o'].isnull()]
    #nos quedamos con los vuelos de 2012
    x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==year]
    x = x[x['mesSalida']==month]
    x = x[x['diaSalida']==day]
    ##cogemos un rango para trabajar con el
    x = x.head(200)################################################################Nº A PROCESAR
    x = x.apply(lambda x:tratar2(x),axis = 1)
    #para escribir ya
    #variable auxiliar

    pd.options.mode.chained_assignment = None  # default='warn'
    vuelosSinTratar['TAVG_o'] = x.set_index(['index'])['TAVG_o'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_o']).values
    vuelosSinTratar['PRCP_o'] = x.set_index(['index'])['PRCP_o'].combine_first(vuelosSinTratar.set_index(['index'])['PRCP_o']).values
    vuelosSinTratar['SNOW_o'] = x.set_index(['index'])['SNOW_o'].combine_first(vuelosSinTratar.set_index(['index'])['SNOW_o']).values
    vuelosSinTratar['TMIN_o'] = x.set_index(['index'])['TMIN_o'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_o']).values
    vuelosSinTratar['TMAX_o'] = x.set_index(['index'])['TMAX_o'].combine_first(vuelosSinTratar.set_index(['index'])['TMAX_o']).values
    vuelosSinTratar['TAVG_d'] = x.set_index(['index'])['TAVG_d'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_d']).values
    vuelosSinTratar['PRCP_d'] = x.set_index(['index'])['PRCP_d'].combine_first(vuelosSinTratar.set_index(['index'])['PRCP_d']).values
    vuelosSinTratar['SNOW_d'] = x.set_index(['index'])['SNOW_d'].combine_first(vuelosSinTratar.set_index(['index'])['SNOW_d']).values
    vuelosSinTratar['TMIN_d'] = x.set_index(['index'])['TMIN_d'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_d']).values
    vuelosSinTratar['TMAX_d'] = x.set_index(['index'])['TMAX_d'].combine_first(vuelosSinTratar.set_index(['index'])['TMAX_d']).values



    vuelos['TAVG_o'] = vuelosSinTratar.set_index(['index'])['TAVG_o'].combine_first(vuelos.set_index(['index'])['TAVG_o']).values
    vuelos['PRCP_o'] = vuelosSinTratar.set_index(['index'])['PRCP_o'].combine_first(vuelos.set_index(['index'])['PRCP_o']).values
    vuelos['SNOW_o'] = vuelosSinTratar.set_index(['index'])['SNOW_o'].combine_first(vuelos.set_index(['index'])['SNOW_o']).values
    vuelos['TMIN_o'] = vuelosSinTratar.set_index(['index'])['TMIN_o'].combine_first(vuelos.set_index(['index'])['TMIN_o']).values
    vuelos['TMAX_o'] = vuelosSinTratar.set_index(['index'])['TMAX_o'].combine_first(vuelos.set_index(['index'])['TMAX_o']).values
    vuelos['TAVG_d'] = vuelosSinTratar.set_index(['index'])['TAVG_d'].combine_first(vuelos.set_index(['index'])['TAVG_d']).values
    vuelos['PRCP_d'] = vuelosSinTratar.set_index(['index'])['PRCP_d'].combine_first(vuelos.set_index(['index'])['PRCP_d']).values
    vuelos['SNOW_d'] = vuelosSinTratar.set_index(['index'])['SNOW_d'].combine_first(vuelos.set_index(['index'])['SNOW_d']).values
    vuelos['TMIN_d'] = vuelosSinTratar.set_index(['index'])['TMIN_d'].combine_first(vuelos.set_index(['index'])['TMIN_d']).values
    vuelos['TMAX_d'] = vuelosSinTratar.set_index(['index'])['TMAX_d'].combine_first(vuelos.set_index(['index'])['TMAX_d']).values
    #vuelos.to_csv('vuelos.csv', sep=',', index=False)
    end = time.time()
    print(end - start)

In [47]:
dia = 1
mes = 1
anyo = 2015

In [84]:
mesw = str(mes)
for dia in range(1,20):
    diaw = str(dia)
    if dia < 10:
        diaw = '0'+diaw
    
    weatherdf = functionWeatherDay(mesw,diaw)
    functionPlus(anyo,mes,dia)


3.52999997139
3.76599979401
3.89599990845
3.52600002289
3.40499997139
3.45399999619
3.34600019455
3.38099980354
3.38000011444
3.5039999485
3.37700009346
3.46099996567
3.34500002861
3.35699987411
94.4569997787
111.551000118
66.9880001545
93.1129999161
112.847000122


In [85]:
vuelos.to_csv('vuelos.csv', sep=',', index=False)

In [86]:
vuelosSinTratar = vuelos[vuelos['TAVG_o'].isnull()]
vuelosTratados = vuelos[vuelos['TAVG_o'].notnull()]
print 'sin tratar de todos ', vuelosSinTratar.shape
print 'tratados ', vuelosTratados.shape
x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==anyo]
print 'sin tratar totales del año ', x.shape
x = x[x['mesSalida']==mes]
print 'del mes sin tratar ', x.shape
x = x[x['diaSalida']==dia]
print 'vuelos del dia sin tratar ' , x.shape

sin tratar de todos  (210814, 66)
tratados  (1884, 66)
sin tratar totales del año  (39692, 66)
del mes sin tratar  (1320, 66)
vuelos del dia sin tratar  (0, 66)


In [87]:
ddd = vuelosTratados['diaSalida'].unique()
ddd.sort()
ddd

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19], dtype=int64)

In [88]:
vuelosTratados.sample(5)

,index,airline_code,flight_number,board_point,board_lat,board_lon,board_country_code,off_point,off_lat,off_lon,off_country_code,distance,scheduled_time_of_departure,actual_time_of_departure,scheduled_time_of_arrival,actual_time_of_arrival,departure_delay,arrival_delay,sched_blocktime,est_blocktime,act_blocktime,aircraft_type,aircraft_registration_number,general_status_code,routing,cabin_1_code,cabin_1_fitted_configuration,cabin_1_saleable,cabin_1_pax_boarded,cabin_1_rpk,cabin_1_ask,cabin_2_code,cabin_2_fitted_configuration,cabin_2_saleable,cabin_2_pax_boarded,cabin_2_rpk,cabin_2_ask,total_rpk,total_ask,load_factor,total_pax,total_no_shows,total_cabin_crew,total_technical_crew,total_baggage_weight,number_of_baggage_pieces,mesSalida,anyoSalida,diaSalida,horaSalida,mesLlegada,anyoLlegada,diaLlegada,horaLlegada,diaSemanaSalida,diaSemanaLlegada,TMIN_o,TMAX_o,TAVG_o,SNOW_o,PRCP_o,TMIN_d,TMAX_d,TAVG_d,SNOW_d,PRCP_d
149343,155179,XX,0,LUX,49.611670,6.130000,LU,LEU,42.338611,1.409167,ES,887,2015-01-04 13:15:00,2015-01-04 13:16:00,2015-01-04 14:50:00,2015-01-04 14:49:00,1,-1,95,NaN,93,320,XJWAT,Departed,LUX-LEU,Y,164,120,97,86015,106411,J,1.0,30.0,15.0,13301.0,26603.0,99317,133013,0.746667,112,0,4,2,1501,99,1,2015,4,13:16:00,1,2015,4,14:49:00,domingo,domingo,9999.0,9999.0,9999.0,9999.0,9999.0,0.3,14.4,8.2,9999.0,0.0
147771,153597,XX,40,BSL,47.589580,7.529910,FR,LEU,42.338611,1.409167,ES,756,2015-01-09 17:35:00,2015-01-09 17:38:00,2015-01-09 18:55:00,2015-01-09 19:00:00,3,5,80,NaN,82,319,XHMAT,Departed,BSL-LEU,Y,137,105,81,61264,79416,J,1.0,22.0,2.0,1513.0,16640.0,62776,96056,0.653543,83,0,3,2,575,38,1,2015,9,17:38:00,1,2015,9,19:00:00,viernes,viernes,4.5,9999.0,9.0,9999.0,1.0,-0.4,17.1,7.4,9999.0,0.0
140256,145503,XX,818,LEU,42.338611,1.409167,ES,AAE,36.900000,7.766670,DZ,813,2015-01-03 10:10:00,2015-01-03 10:06:00,2015-01-03 12:35:00,2015-01-03 12:36:00,-4,1,145,NaN,150,320,XJRAT,Departed,LEU-AAE,Y,165,165,42,34158,134193,NaN,NaN,NaN,NaN,NaN,NaN,34158,134193,0.254545,42,0,4,2,693,46,1,2015,3,10:06:00,1,2015,3,12:36:00,s�bado,s�bado,-2.5,16.0,4.7,9999.0,0.0,9999.0,9999.0,9999.0,9999.0,9999.0
177573,184264,QN,6699,LIG,45.831530,1.257800,FR,LEU,42.338611,1.409167,ES,389,2015-01-02 07:30:00,2015-01-02 07:49:00,2015-01-02 09:30:00,2015-01-02 09:41:00,19,11,120,NaN,112,AT7,MFRAT,Departed,LIG-LEU,Y,68,68,19,7383,26424,NaN,NaN,NaN,NaN,NaN,NaN,7383,26424,0.279412,19,0,2,2,315,19,1,2015,2,07:49:00,1,2015,2,09:41:00,viernes,viernes,-1.4,5.4,1.9,9999.0,0.0,-4.5,14.7,2.9,9999.0,0.0
193329,201042,XX,762,SYY,58.209250,-6.386490,GB,LEU,42.338611,1.409167,ES,1847,2015-01-19 10:20:00,2015-01-19 10:30:00,2015-01-19 14:15:00,2015-01-19 14:34:00,10,19,175,NaN,184,321,XLRAT,Departed,SYY-LEU,Y,194,172,155,286242,317637,J,2.0,16.0,13.0,24007.0,29548.0,310250,347184,0.893617,168,0,4,2,2583,161,1,2015,19,10:30:00,1,2015,19,14:34:00,lunes,lunes,9999.0,9999.0,0.1,9999.0,0.0,1.4,7.5,3.7,9999.0,0.0


#############PRUEBAS###########################

In [ ]:
#en este punto debemos pasarle la lat y lon del vuelo

a= closest(np.asarray(lats), [45.800000, 24.150000])
aux = stationsdf[stationsdf['lat']==a[0]]
aux = stationsdf[stationsdf['lng']==a[1]]
stationid = aux['id']
s =stationid.to_string()
stationid = str(s)
stationid = stationid.split()[1]
fecha = tratarFecha('2016-07-18 21:51:00')

In [116]:
del weatherdf['Value3']
del weatherdf['Value4']
del weatherdf['Value5']
#del weatherdf['Value2']

In [182]:
weatherdf[weatherdf['date']-20150000==101]

,id,date,type,Value1
0,US1NJGL0001,20150101,PRCP,0
1,US1NJGL0001,20150101,SNOW,0
2,ASN00015643,20150101,TMAX,373
3,ASN00015643,20150101,TMIN,222
4,ASN00015643,20150101,PRCP,0
5,US1NDWL0002,20150101,PRCP,0
6,US1NDWL0002,20150101,SNOW,0
7,US1NCTR0002,20150101,PRCP,0
8,US1NCTR0002,20150101,SNOW,0
9,ASN00085296,20150101,TMAX,254


In [ ]:
#Now we extract the data from the climate table
datatavg = weatherdf[(weatherdf["id"]=='USW00094860')].sort_values("date")
datatavg["Value1"]=datatavg["Value1"]/10
datatavg = datatavg.reset_index()
datatavg

In [ ]:
 #primero tratamos la fecha
fecha = tratarFecha('20161023')
coor = [45.080689,3.762889]
a= closest(np.asarray(lats), coor)
aux = stationsdf[stationsdf['lat']==a[0]]
aux = stationsdf[stationsdf['lng']==a[1]]
stationid = aux['id']
s =stationid.to_string()
stationid = str(s)
stationid = stationid.split()[1]
    
datatavg = weatherdf[(weatherdf["id"]==stationid)&(weatherdf['date']==fecha)].sort_values("date")
datatavg["Value1"]=datatavg["Value1"]/10
datatavg = datatavg.reset_index()
T = datatavg[datatavg['type']=='PRCP']['Value1']
if T.empty:
    T = 0.0
    return T
else:
    return T

In [112]:
start = time.time()
closest(np.asarray(lats), [45.800000, 24.150000])
end = time.time()
print(end - start)

0.323999881744


In [ ]:
vuelos['actual_time_of_departure']

In [ ]:
var2 = '20160515'
var2[:4]



In [ ]:
x = tratarFecha('2016-05-15 11:41:00')

In [ ]:
str(x)

In [ ]:
vuelos2016.shape

In [ ]:
vuelos2016_01 = vuelos2016[vuelos2016['mesSalida']==1]
vuelos2016_02 = vuelos2016[vuelos2016['mesSalida']==2]


In [ ]:
print vuelos2016_01.shape
print vuelos2016_02.shape

In [ ]:
7*12*31

In [ ]:
def tratar(param,lat, lon, fecha):
    #primero tratamos la fecha
    fecha = tratarFecha(fecha)
    coor = [lat,lon]
    a= closest(np.asarray(lats), coor)
    aux = stationsdf[stationsdf['lat']==a[0]]
    aux = stationsdf[stationsdf['lng']==a[1]]
    stationid = aux['id']
    s =stationid.to_string()
    stationid = str(s)
    stationid = stationid.split()[1]
    datatavg = weatherdf[(weatherdf["id"]==stationid)&(weatherdf['date']==fecha)].sort_values("date")
    datatavg["Value1"]=datatavg["Value1"]/10
    datatavg = datatavg.reset_index()
    T = datatavg[datatavg['type']==param]['Value1']
    if T.empty:
        T = 9999
    return float(T)

In [ ]:
#metodo antiguo
#vuelos2014["TAVG_o"] = vuelos2014.apply(lambda vuelos2014: tratar('TAVG',vuelos2014['board_lat'],vuelos2014['board_lon'],vuelos2014['actual_time_of_departure']), axis = 1)
#vuelos2014["PRCP_o"] = vuelos2014.apply(lambda vuelos2014: tratar('PRCP',vuelos2014['board_lat'],vuelos2014['board_lon'],vuelos2014['actual_time_of_departure']), axis = 1)
#vuelos2014["SNOW_o"] = vuelos2014.apply(lambda vuelos2014: tratar('SNOW',vuelos2014['board_lat'],vuelos2014['board_lon'],vuelos2014['actual_time_of_departure']), axis = 1)
#vuelos2014["TMIN_o"] = vuelos2014.apply(lambda vuelos2014: tratar('TMIN',vuelos2014['board_lat'],vuelos2014['board_lon'],vuelos2014['actual_time_of_departure']), axis = 1)
#vuelos2014["TMAX_o"] = vuelos2014.apply(lambda vuelos2014: tratar('TMAX',vuelos2014['board_lat'],vuelos2014['board_lon'],vuelos2014['actual_time_of_departure']), axis = 1)

In [42]:
mes = 1
dia = 1
mesw = str(mes)
diaw = str(dia)
if dia < 10:
    diaw = '0'+diaw

In [45]:
diaw
 

'01'

In [46]:
mesw

'1'